In [75]:
import os
from keras.models import Model, load_model
import keras.losses
import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np

import time


def custom_loss(y_true, y_predicted):
    mean, log_sigma = tf.split(y_predicted, 2, axis=-1)
    y_true_value, temp = tf.split(y_true, 2, axis=-1)
    sigma = tf.nn.softplus(log_sigma)
    dist = tfp.distributions.MultivariateNormalDiag(loc=mean, scale_diag=sigma)
    loss = -tf.reduce_mean(dist.log_prob(y_true_value))
    return loss


root_path = f'/home/yigit/phd/yigit_phd_thesis/cnmp/output/sfm/1_obs_huge/'
test_id = '1604977395'
model_path = f'{root_path}{test_id}/'

data_root = "/home/yigit/phd/yigit_phd_thesis/cnmp/data/sfm/1_obs_huge/"
data_path = f"{data_root}demonstrations/"
output_path = f'{data_root}/tsne/'

keras.losses.custom_loss = custom_loss
model = load_model(f'{model_path}cnmp_best_validation.h5', custom_objects={'tf': tf})

latent_layer = Model(inputs=model.input, outputs=model.get_layer('obs_mlp').get_output_at(-1))

In [76]:
def select_data(X, Y, G):
    idx_left = np.where(np.logical_and(gamma[:,0,0] > 1.97, gamma[:,0,0] < 2.03))[0][0]
    idx_right = np.where(np.logical_and(gamma[:,0,0] > -2.03, gamma[:,0,0] < -1.97))[0][0]
    
    x=np.zeros((2, X.shape[1], X.shape[2]))
    y=np.zeros((2, Y.shape[1], Y.shape[2]))
    g=np.zeros((2, G.shape[1], G.shape[2]))
    
    x[0] = X[idx_left]
    x[1] = X[idx_right]
    y[0] = Y[idx_left]
    y[1] = Y[idx_right]
    g[0] = G[idx_left]
    g[1] = G[idx_right]
    
    return x, y, g

In [77]:
all_X, all_Y, all_gamma = (np.load(data_path + 'd_x.npy'), np.load(data_path + 'd_y.npy'), np.load(data_path + 'd_gamma.npy'))

X, Y, gamma = all_X, all_Y, all_gamma
# X, Y, gamma = select_data(all_X, all_Y, all_gamma)


In [78]:
d_N = X.shape[0]
d_x, d_y, d_gamma = (X.shape[-1], Y.shape[-1], gamma.shape[-1])  # d_x, d_y: dimensions
time_len = X.shape[1]

print(f'd_N = {d_N}')
print(f'd_x = {d_x}')
print(f'd_y = {d_y}')
print(f'd_gamma = {d_gamma}')
print(f'time_len = {time_len}')

d_N = 2300
d_x = 2
d_y = 2
d_gamma = 2
time_len = 400


In [93]:
def get_tsne_sample(index, n):
    # select an observation and n targets
    target_step = -1
    observation = np.zeros((1, n, d_x + d_y + d_gamma))
    target_X_gamma = np.zeros((1, 1, d_x + d_gamma))
    
    perm = np.random.permutation(time_len)
    observation[0, :n, :d_x] = X[index, perm[:n]]
    observation[0, :n, d_x:d_x+d_gamma] = gamma[index, perm[:n]]  # gamma concatenation to observation
    observation[0, :n, d_x+d_gamma:d_x+d_gamma+d_y] = Y[index, perm[:n]]
    
    # last step used as target (not actually used but just for compatibility issues)
    target_X_gamma[0, 0, :d_x] = X[index, target_step]
    target_X_gamma[0,:n, d_x:d_x+d_gamma] = gamma[index, target_step]  # gamma concatenation to target query
    return (observation, target_X_gamma)

In [94]:
a, b = get_tsne_sample(0, 2)  # testing method
print(a.shape)
print(b.shape)

(1, 2, 6)
(1, 1, 4)


In [95]:
latent_shape = (1, 1, 64)
n = 100  # number of samples from each trajectory
latent_training_values = np.zeros((d_N * n, latent_shape[2]))  # for each of d_N trajectories, we have n samples

for i in range(d_N):
    observations, target = get_tsne_sample(i, n)
    for j in range(n):
        observation = observations[:, j, :].reshape(1, 1, d_x+d_gamma+d_y)
        latent_training_values[i*n+j] = latent_layer.predict([observation, target]).reshape(1, latent_shape[2])

# obs_ = np.array([0.0831583e-02, 2.8e+01, 1.006e+00, 14.0, 0.0, 2.01129232e-04]).reshape(1, 1, 6)
# tar_ = np.array([0, 0, 0, 0]).reshape(1, 1, 4)
# obs_ = np.array([1, 10, -0.25, -0.25, 1, 1]).reshape(1, 1, 6)
# tar_ = np.array([0, 0, 1, 0]).reshape(1, 1, 4)
# latent_values[-4] = latent_layer.predict([obs_, tar_]).reshape(1, latent_shape[2])
# obs_ = np.array([0, 0, 0, 0.0, 0.0, 0]).reshape(1, 1, 6)
# tar_ = np.array([0, 0, 0, 0]).reshape(1, 1, 4)
# latent_values[-3] = latent_layer.predict([obs_, tar_]).reshape(1, latent_shape[2])
# obs_ = np.array([100, 100, 100, 100, 100, 100]).reshape(1, 1, 6)
# tar_ = np.array([100, 100, 100, 100]).reshape(1, 1, 4)
# latent_values[-2] = latent_layer.predict([obs_, tar_]).reshape(1, latent_shape[2])
# obs_ = np.array([-100, -100, -100, -100, -100, -100]).reshape(1, 1, 6)
# tar_ = np.array([-100, -100, -100, -100]).reshape(1, 1, 4)
# latent_values[-1] = latent_layer.predict([obs_, tar_]).reshape(1, latent_shape[2])

np.save(f'{output_path}{test_id}.npy', latent_training_values)

In [ ]:
pca_flag = False
if not pca_flag:
    from sklearn.manifold import TSNE

    tsne = TSNE(n_jobs=-1)
    latent_tsne = tsne.fit_transform(latent_training_values)
    print(latent_tsne.shape)
else:
    from sklearn.decomposition import PCA

    pca = PCA(n_components=2)
    latent_pca = pca.fit_transform(latent_training_values)
    print(latent_pca.shape)

In [ ]:
import matplotlib.pyplot as plt

if not pca_flag:
    plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], s=0.99)
#     plt.scatter(latent_tsne[0:n, 0], latent_tsne[0:n, 1], s=1, color='black')
#     plt.scatter(latent_tsne[n:, 0], latent_tsne[n:, 1], s=1)
#     plt.plot(latent_tsne[-4, 0], latent_tsne[-4, 1],'rx')
#     plt.plot(latent_tsne[-3, 0], latent_tsne[-3, 1],'yo')
#     plt.plot(latent_tsne[-2, 0], latent_tsne[-2, 1],'ks')
#     plt.plot(latent_tsne[-1, 0], latent_tsne[-1, 1],'rx')
else:
    plt.scatter(latent_pca[:, 0], latent_pca[:, 1], s=0.99)
    # plt.scatter(latent_pca[0:n, 0], latent_pca[0:n, 1], s=0.99, color='black')
    # plt.scatter(latent_pca[n:-4, 0], latent_pca[n:-4, 1], s=0.99)
    # plt.plot(latent_pca[-4, 0], latent_pca[-4, 1],'rx')
    # plt.plot(latent_pca[-3, 0], latent_pca[-3, 1],'yo')
    # plt.plot(latent_pca[-2, 0], latent_pca[-2, 1],'k.')
    # plt.plot(latent_pca[-1, 0], latent_pca[-1, 1],'rx')
plt.show()

In [ ]:
runs_dir = '/home/yigit/phd/yigit_phd_thesis/cnmp/output/sfm/1_obs_huge/1604302554/runs/'
latent_states = []
for run in os.listdir(runs_dir):
    tmp_run = np.load(os.path.join(runs_dir, run))
    len_tmp_run = len(tmp_run)
    target_X_gamma = np.array([tmp_run[-1][0], tmp_run[-1][1], tmp_run[-1][2], tmp_run[-1][3]]).reshape(1, 1, d_x+d_gamma)  # x|gamma
    for i in range(1, len_tmp_run):  # first steps are wrong
        observation = tmp_run[i].reshape(1, 1, d_x+d_gamma+d_y)
        latent_states.append(latent_layer.predict([observation, target_X_gamma]).reshape(1, latent_shape[2]))

latent_states = np.array(latent_states).reshape(len(latent_states), latent_shape[2])

if not pca_flag:
    latent_states_tsne = tsne.transform(latent_states)
    plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], s=0.99)
    plt.scatter(latent_states_tsne[:, 0], latent_states_tsne[:, 1], s=0.99, color='red')
else:
    #combined_pca = pca.fit_transform(combined)
    latent_states_pca = pca.transform(latent_states)
    plt.scatter(latent_pca[:, 0], latent_pca[:, 1], s=0.99)
    plt.scatter(latent_states_pca[:, 0], latent_states_pca[:, 1], s=0.99, color='red')
plt.show()